In [1]:
import tensorflow as tf
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from skimage.transform import resize
from my_utils import *

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
# read the all names of files that contains person
# get the path of masks and the path of data
person_names_mat = os.listdir('VOC2010/all_person_masks')
person_names_jpg = []
for i in range(len(person_names_mat)):
    temp = person_names_mat[i][:-4]+'.jpg'
    person_names_jpg.append(temp)
mat_path = []
for i in range(len(person_names_mat)):
    path = os.path.join('VOC2010/all_person_masks',person_names_mat[i])
    mat_path.append(path)
jpg_path = []
for i in range(len(person_names_jpg)):
    path = os.path.join('VOC2010/JPEGImages',person_names_jpg[i])
    jpg_path.append(path)
data_table = pd.DataFrame(jpg_path,columns=['jpg_path'])
mat_path_df = pd.DataFrame(mat_path,columns=['mat_path'])
data_table = pd.concat([data_table,mat_path_df],axis=1)
data_table.head()


,jpg_path,mat_path
0,VOC2010/JPEGImages\2008_000003.jpg,VOC2010/all_person_masks\2008_000003.mat
1,VOC2010/JPEGImages\2008_000008.jpg,VOC2010/all_person_masks\2008_000008.mat
2,VOC2010/JPEGImages\2008_000023.jpg,VOC2010/all_person_masks\2008_000023.mat
3,VOC2010/JPEGImages\2008_000026.jpg,VOC2010/all_person_masks\2008_000026.mat
4,VOC2010/JPEGImages\2008_000032.jpg,VOC2010/all_person_masks\2008_000032.mat


In [ ]:
# read and preprosess the data
# read img
Data = []
for i in range(30):
    test_img = image.load_img(data_table['jpg_path'][i])
    test_img = image.img_to_array(test_img)
    # read mask
    test_mask = scipy.io.loadmat(data_table['mat_path'][i])['single_map']
    crop1,crop2,crop3 = random_put_into_256map(test_img,test_mask)
    Data.append(crop1)
    Data.append(crop2)
    Data.append(crop3)

In [ ]:
# split the data into train and validation
train_data = Data[0:70]
val_data = Data[70:90]


## ---test---

In [28]:
img,mask = get_next_batch(train_data)
plt.figure()
plt.imshow(img/255)
plt.show()
plt.figure()
plt.imshow(mask)
plt.show()
aug_img,aug_mask = augmentor_with_keras(img,mask)
plt.figure()
plt.imshow(aug_img/255)
plt.show()
plt.figure()
plt.imshow(aug_mask)
plt.show()
scale_mask = my_scale(aug_mask)
plt.figure()
plt.imshow(scale_mask)
plt.show()

## ---test end---


In [ ]:
def process_config(conf_file):
	"""
	"""
	params = {}
	config = configparser.ConfigParser()
	config.read(conf_file)
	for section in config.sections():
		if section == 'DataSetHG':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Network':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Train':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Validation':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
		if section == 'Saver':
			for option in config.options(section):
				params[option] = eval(config.get(section, option))
	return params
print('--Parsing Config File')
params = process_config('config.cfg')

In [ ]:
model = HourglassModel(nFeat=params['nfeats'], nStack=params['nstacks'], 
                       nModules=params['nmodules'], 
                       nLow=params['nlow'], outputDim=params['num_joints'], 
                       batch_size=params['batch_size'], attention = params['mcam'],
                       training=True, drop_rate= params['dropout_rate'], 
                       lear_rate=params['learning_rate'], 
                       decay=params['learning_rate_decay'], 
                       decay_step=params['decay_step'], 
                       name=params['name'], 
                       logdir_train=params['log_dir_train'], 
                       logdir_test=params['log_dir_test'], 
                       tiny= params['tiny'], w_loss=params['weighted_loss'] , 
                       joints= params['joint_list'],modif=False)
model.generate_model()

In [ ]:
# output: (None,8,64,64,15)
output = model.get_output()
loss = model.loss
out = output[0,7]
pred_seg = tf.argmax(out,axis=2)
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# train
iteration = []
acc_list = []
iou_list = []
for i in range(10):
    x_b,y_b = get_next_batch(train_data,train_label)
    x,y = get_augmentor_batch(x_b,y_b,3)
    sess.run(train_step,feed_dict={model.img:x, model.gtMaps:y})

    x_for_eval = np.expand_dims(x_b,axis=0)
    pred = sess.run(pred_seg,feed_dict={model.img:x_for_eval})
    acc = cal_acc(pred,y_b)
    iou = cal_iou(pred,y_b)
    print('acc:%f',acc,'  ','iou:%f',iou)
    iteration.append(i)
    acc_list.append(acc)
    iou_list.append(iou)

In [ ]:
# save iou and acc
eval_info = pd.DataFrame([iteration,acc_list,iou_list],columns=['iter','accracy','iou'])
eval_info.to_csv('training_info_with_pascal.csv')

In [ ]:
# save the network
path = "my_net_with_pascal"
is_exist = os.path.exists(path)
if not is_exist:
    os.makedirs(path)
full_path = os.path.join(path,'save_net.ckpt')
save_path = saver.save(sess,full_path)